## 추천시스템

### 학습 목표
- 추천 시스템의 개념과 목적을 이해한다.
- Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.
- 추천 시스템이 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다.
- 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.

### 루브릭 
-  CSR matrix가 정상적으로 만들어졌다. : **사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.**
- MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다. : **사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.**
- 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다. : **MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.**

## 목차
### 1. 데이터 준비와 전처리
#### 1) 데이터 로드
#### 2) 데이터 분석
#### 3) 내가 좋아하는 영화 추가 및 indexing
### 2. 모델 구성
#### 1) CSR matrix
#### 2) Alternating Least Squares 모델
### 3. 모델 평가
#### 1) 좋아하는 영화와 비슷한 영화
#### 2) 내가 좋아할 만한 영화
---

## 1. 데이터 준비와 전처리
### 1) 데이터 로드

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

별점(ratings)을 시청횟수(counts)로 바꿈으로써 별점 데이터를 explicit 데이터에서 implicit 데이터로 간주한다.

위에서 3점 이상만을 남겨두었기 때문에, 유저가 3번 이상 본 영화는 선호한다고 판단하고, 많이 본 영화에 대해 가중치를 주어 더 확실히 좋아한다고 판단하는 규칙을 적용한다.

In [4]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings = pd.merge(left = ratings, right = movies, how = 'left', on = 'movie_id')
ratings.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


### 2) 데이터 분석

In [6]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [7]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [8]:
# 시청 횟수를 모두 세어 가장 많은 것을 인기 있는 영화라고 가정하고 30개 선정
movie_count = ratings.groupby('title')['counts'].sum()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   14449
Star Wars: Episode IV - A New Hope (1977)                13178
Star Wars: Episode V - The Empire Strikes Back (1980)    12648
Saving Private Ryan (1998)                               11348
Star Wars: Episode VI - Return of the Jedi (1983)        11303
Raiders of the Lost Ark (1981)                           11179
Silence of the Lambs, The (1991)                         11096
Matrix, The (1999)                                       10903
Sixth Sense, The (1999)                                  10703
Terminator 2: Judgment Day (1991)                        10513
Fargo (1996)                                             10465
Schindler's List (1993)                                  10317
Braveheart (1995)                                        10125
Shawshank Redemption, The (1994)                         10085
Back to the Future (1985)                                10081
Godfather, The (1972)                            

In [9]:
# 유저별 몇 개의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

### 3) 내가 좋아하는 영화 추가 및 indexing

In [10]:
my_favorite = ['Sound of Music, The (1965)', 
               'Titanic (1997)', 
               'Matrix, The (1999)', 
               'Shakespeare in Love (1998)', 
               'Phantom of the Opera, The (1943)']

my_playlist = pd.DataFrame({'user_id': ['jieun']*5, 'counts': [5]*5, 'title': my_favorite})

if not ratings.isin({'user_id':['jieun']})['user_id'].any():  
    ratings = ratings.append(my_playlist)                       

ratings.tail(10)

,user_id,movie_id,counts,timestamp,title,genre
836473,6040,1090.0,3,956715518.0,Platoon (1986),Drama|War
836474,6040,1094.0,5,956704887.0,"Crying Game, The (1992)",Drama|Romance|War
836475,6040,562.0,5,956704746.0,Welcome to the Dollhouse (1995),Comedy|Drama
836476,6040,1096.0,4,956715648.0,Sophie's Choice (1982),Drama
836477,6040,1097.0,4,956715569.0,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
0,jieun,NaN,5,NaN,"Sound of Music, The (1965)",NaN
1,jieun,NaN,5,NaN,Titanic (1997),NaN
2,jieun,NaN,5,NaN,"Matrix, The (1999)",NaN
3,jieun,NaN,5,NaN,Shakespeare in Love (1998),NaN
4,jieun,NaN,5,NaN,"Phantom of the Opera, The (1943)",NaN


영화들 중 좋아하는 영화 다섯개를 추가한다. 'Sound of Music, The (1965)', 'Titanic (1997)', 'Matrix, The (1999)', 'Shakespeare in Love (1998)', 'Phantom of the Opera, The (1943)'

In [11]:
user = ratings['user_id'].unique()
movie = ratings['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user)}
movie_to_idx = {v:k for k,v in enumerate(movie)}

In [12]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
ratings['user_id'] = temp_user_data


temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
ratings['movie_id'] = temp_movie_data

ratings

,user_id,movie_id,counts,timestamp,title,genre
0,0,0,5,978300760.0,One Flew Over the Cuckoo's Nest (1975),Drama
1,0,1,3,978302109.0,James and the Giant Peach (1996),Animation|Children's|Musical
2,0,2,3,978301968.0,My Fair Lady (1964),Musical|Romance
3,0,3,4,978300275.0,Erin Brockovich (2000),Drama
4,0,4,5,978824291.0,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...
0,6039,14,5,NaN,"Sound of Music, The (1965)",NaN
1,6039,27,5,NaN,Titanic (1997),NaN
2,6039,124,5,NaN,"Matrix, The (1999)",NaN
3,6039,126,5,NaN,Shakespeare in Love (1998),NaN


## 2. 모델 구성
### 1) CSR matrix

In [13]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_artist = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_artist))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

user의 수 × 영화의 수 CSR matrix는 한 유저가 하나의 행을 의미하고, 그 유저가 각 영화를 본 횟수(counts)를 각 영화의 열에 표기한 형태로 구성되어 있다. 유저들은 평균 약 138개, 최대 1968개의 영화를 보았으므로, 해당 행렬은 대부분이 비어있는 sparse 행렬이다.

### 2) Alternating Least Squares 모델

In [14]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [15]:
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

AlternatingLeastSquares 클래스의 __init__ 파라미터

- factors : 유저와 아이템 벡터의 차원 수
- regularization : 과적합을 방지하기 위한 정규화 값
- use_gpu : GPU를 사용할 것인지 여부
- iterations : epochs

In [16]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [17]:
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

In [18]:
jieun = user_to_idx['jieun']
sound_of_music = movie_to_idx['Sound of Music, The (1965)']

jieun_vector = als_model.user_factors[jieun]
sound_of_music_vector = als_model.item_factors[sound_of_music]

In [19]:
jieun_vector

array([ 0.14499576, -0.91826206, -0.28097576, -0.45957837,  0.00255883,
       -0.07329886,  0.33347854, -0.22390737, -0.25311518,  0.05010381,
        0.4175642 , -0.09067797, -0.29967704,  0.47943562,  0.26494226,
       -0.54674983,  0.55077374, -0.0635194 , -0.11177675,  0.79039305,
       -0.35232818,  0.2529134 ,  0.19460484, -0.54410505,  0.05086036,
       -0.17281856, -0.348921  ,  0.6846784 ,  0.01078254,  0.3406949 ,
        0.0516992 , -0.01220614,  0.18568923, -0.1704834 ,  0.06660678,
        0.17654203,  0.25958395, -0.03840999,  0.06418176, -0.7673723 ,
       -0.16608965,  0.7060228 , -0.00903401,  0.04564879, -0.26020712,
        0.07478253,  0.17894416,  0.22261673,  0.48151392,  0.49177784,
        0.35997203,  0.41383052, -0.05324512,  0.07695308,  0.46573502,
       -0.04270403, -0.822548  ,  0.2201753 , -0.06530973, -0.20103864,
       -0.10684951,  0.16288018,  0.15812074,  0.298798  ,  0.92053866,
       -0.82691854, -0.26641688, -0.1263432 , -0.07320342, -0.14

In [20]:
sound_of_music_vector

array([ 0.0297441 , -0.00162377,  0.02549857,  0.00256294,  0.00904942,
        0.00876096,  0.00426919,  0.00542114,  0.00283762,  0.00097158,
       -0.00641226, -0.00473189,  0.01617435,  0.02361163,  0.01559885,
        0.00552973,  0.02888905,  0.00588196, -0.01078974,  0.00871321,
        0.00581265,  0.00876994,  0.01674856, -0.03089322, -0.00727907,
        0.0159946 ,  0.01801036,  0.0140858 ,  0.0332532 ,  0.01241449,
       -0.01772506,  0.02806432,  0.00071675,  0.02166759,  0.00817804,
        0.01704357, -0.0041297 ,  0.00778558, -0.01276015, -0.00875155,
       -0.00717255, -0.00245281, -0.02323906, -0.01024333,  0.00746371,
       -0.00982755,  0.02284358,  0.01831575,  0.01719859,  0.01231764,
        0.01145321,  0.02629594, -0.0025436 ,  0.02630954,  0.02777888,
        0.00396786,  0.00390015,  0.00393338, -0.00866775,  0.00070122,
        0.02654677,  0.04152778,  0.0029701 ,  0.01862693,  0.00516296,
       -0.00497352, -0.03033042, -0.0068901 ,  0.00208642,  0.01

In [21]:
np.dot(jieun_vector, sound_of_music_vector)

0.56314456

'jieun' 벡터와 'sound of music' 벡터를 내적한 결과 약 0.56 수준의 값이 도출되었다.

## 3. 모델 평가
### 1) 좋아하는 영화와 비슷한 영화

In [23]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

In [24]:
def get_similar_movie(movie_code):
    movie_id = movie_to_idx[movie_code]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [25]:
get_similar_movie('Sound of Music, The (1965)')

['Sound of Music, The (1965)',
 'Oliver! (1968)',
 'My Fair Lady (1964)',
 'King and I, The (1956)',
 'West Side Story (1961)',
 'Gigi (1958)',
 'Mary Poppins (1964)',
 'White Christmas (1954)',
 'James and the Giant Peach (1996)',
 'Phantom of the Opera, The (1943)']

모델은 sound of music과 비슷한 영화로 My fair lady, West Side Story 등 뮤지컬 영화를 꼽고 있다. 

### 2) 내가 좋아할 만한 영화

In [26]:
user = user_to_idx['jieun']

movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
[idx_to_movie[i[0]] for i in movie_recommended]

['Terminator 2: Judgment Day (1991)',
 'Fantastic Voyage (1966)',
 'Men in Black (1997)',
 'Sixth Sense, The (1999)',
 'Gone with the Wind (1939)',
 'Saving Private Ryan (1998)',
 'American Beauty (1999)',
 'My Fair Lady (1964)',
 'Full Monty, The (1997)',
 'Voyage to the Bottom of the Sea (1961)',
 'Edward Scissorhands (1990)',
 'Creature From the Black Lagoon, The (1954)',
 'Abbott and Costello Meet Frankenstein (1948)',
 '2001: A Space Odyssey (1968)',
 'Dark City (1998)',
 'West Side Story (1961)',
 'King and I, The (1956)',
 'Thomas Crown Affair, The (1999)',
 'Speed (1994)',
 'Oliver! (1968)']

모델은 'jieun'에게 'sound of music'과 비슷하다고 판단한 'King and I', 'Oliver!', 'My Fair Lady' 등과 함께 'Terminator 2', 'Saving Private Ryan' 등을 함께 추천했다.

In [28]:
Terminator = movie_to_idx['Terminator 2: Judgment Day (1991)']
explain = als_model.explain(user, csr_data, itemid=Terminator)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Matrix, The (1999)', 0.3209163737761338),
 ('Sound of Music, The (1965)', 0.04856718293598784),
 ('Titanic (1997)', 0.028594689401168376),
 ('Phantom of the Opera, The (1943)', 0.01982498146680561),
 ('Shakespeare in Love (1998)', 0.0013648508406580715)]

'Terminator 2'의 추천에 기여한 정도를 보면 'Matrix'의 기여가 다른 항목에 비해 지나치게 높다는 인상이 든다.

In [30]:
ter_vector = als_model.item_factors[Terminator]
np.dot(jieun_vector, ter_vector)

0.4240936

'jieun' 벡터와 'Terminator 2' 벡터를 내적한 결과는 약 0.42 수준으로 역시 높은 편이다.

In [31]:
get_similar_movie('Matrix, The (1999)')

['Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Terminator, The (1984)',
 'Fifth Element, The (1997)',
 'Fugitive, The (1993)',
 'Face/Off (1997)',
 'Female Perversions (1996)',
 'Twelve Monkeys (1995)',
 'Mrs. Dalloway (1997)']

'Matrix'와 유사한 영화로 'Terminator 2'가 꼽히는 것은 맞지만, 모델이 'Matrix'와 유사한 영화에 치우쳐서 추천하지는 않은 것을 확인할 수 있다.

## 회고

- 너무 옛날 영화들이라... 거의 본 영화가 드물어서 좋아하는 영화를 찾기 곤란하다고 느낌과 동시에, 이와 같이 유저가 낯선 상품(아이템)들 속에서 자신의 취향을 별점 등으로 명확하게 표현하지 못할 경우가 있으므로 별점('ratings')과 같은 명시적 평가를 '재생횟수'라는 암묵적 평가로 바꾸는 것이 효과가 있을 것이라는 생각이 들었다.
- 'terminator 2'를 추천할 때 'Matrix' 한편의 기여도가 지나치게 높다고 생각했지만, 좋아하는 영화 5편 모두의 기여도가 유사하게끔 한다면 오히려 아무런 작품도 추천하지 못할 것이다. ('Sound of music'과 'Matrix' 둘 다와 닮은 영화가...흔치는 않을테니..) 이런 면에서 다소 '한 사람이 가진 모든 취향에 맞출 수는 없다'는 점을 감안하고 설계되는 것이 추천시스템이 아닐까 싶다.
- 사람이 남긴 흔적 데이터를 가지고 사람이나 사회 현상이 어떤 양상을 띄는지 해석하고 예측하는 것은 데이터를 이용해 가장 하고 싶은 일이니만큼 추천시스템은 기회가 된다면 좀 더 깊이있게 공부해보고 싶다.